In [1]:
#!/usr/bin/python

from __future__ import print_function
import math
import time,random,threading
import tensorflow as tf
from time import sleep
import numpy as np
from vizdoom import *
import skimage.color, skimage.transform
from tqdm import tqdm
from tensorflow.python import debug as tf_debug

CONFIG_FILE_PATH = "./config/custom_config.cfg"
MODEL_PATH = "./model_v00/model_v00.ckpt"
RESOLUTION = (120,180,3)

N_ADV = 5

UPDATE_FREQ = 10

N_ACTION = 6

GAMMA = 0.99

BOTS_NUM = 5

REWARDS = {'living':-0.01, 'health_loss':-1, 'medkit':50, 'ammo':0.0, 'frag':500, 'dist':3e-2, 'suicide':-500} 

LEARNING_RATE = 5e-3
RMSProbDecaly = 0.99

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class TestEnvironment(object):
    def __init__(self,name, parameter_server):
        self.game = DoomGame()
        self.game.load_config(CONFIG_FILE_PATH)
        self.game.set_window_visible(True)
        self.game.set_mode(Mode.PLAYER)
#         self.game.set_screen_format(ScreenFormat.GRAY8)
        self.game.set_screen_format(ScreenFormat.CRCGCB)
        self.game.set_screen_resolution(ScreenResolution.RES_640X480)
        self.game.init()
        
        health = self.game.get_game_variable(GameVariable.HEALTH)
        ammo = self.game.get_game_variable(GameVariable.SELECTED_WEAPON_AMMO)
        frag = self.game.get_game_variable(GameVariable.FRAGCOUNT)
        pos_x = self.game.get_game_variable(GameVariable.POSITION_X)
        pos_y = self.game.get_game_variable(GameVariable.POSITION_Y)
        self.reward_gen = RewardGenerater(health,ammo,frag,pos_x,pos_y)
        
        self.network = Network_local(name, parameter_server)
        
        self.agent = Agent(name,self.network)

        
        self.pre_death = 0
    
    def start_episode(self):
        self.game.new_episode()
        for i in range(BOTS_NUM):
            self.game.send_game_command("addbot")

        
    def preprocess(self,img):
        if len(img.shape) == 3:
            img = img.transpose(1,2,0)

        img = skimage.transform.resize(img, RESOLUTION,mode='constant')
        img = img.astype(np.float32)
        return img
    
    def get_reward(self):
        health = self.game.get_game_variable(GameVariable.HEALTH)
        ammo = self.game.get_game_variable(GameVariable.SELECTED_WEAPON_AMMO)
        frag = self.game.get_game_variable(GameVariable.FRAGCOUNT)
        pos_x = self.game.get_game_variable(GameVariable.POSITION_X)
        pos_y = self.game.get_game_variable(GameVariable.POSITION_Y)
        
        r,r_detail = self.reward_gen.get_reward(health,ammo,frag,pos_x,pos_y)
    
        return r
    
    def run(self):

        global frames
        health = self.game.get_game_variable(GameVariable.HEALTH)
        ammo = self.game.get_game_variable(GameVariable.SELECTED_WEAPON_AMMO)
        frag = self.game.get_game_variable(GameVariable.FRAGCOUNT)
        pos_x = self.game.get_game_variable(GameVariable.POSITION_X)
        pos_y = self.game.get_game_variable(GameVariable.POSITION_Y)
        self.reward_gen = RewardGenerater(health,ammo,frag,pos_x,pos_y)
        
        self.start_episode()
        
        #Copy params from global
        self.agent.network.pull_parameter_server()

        step = 0
        while not self.game.is_episode_finished():
            
            if step%N_ADV==0 and not step==0:
                self.reward_gen.update_origin(self.game.get_game_variable(GameVariable.POSITION_X),\
                                              self.game.get_game_variable(GameVariable.POSITION_Y))

            s1 = self.preprocess(self.game.get_state().screen_buffer)
            action = self.agent.test_act(s1)
            self.game.make_action(action,1)
            reward = self.get_reward()
            isterminal = self.game.is_episode_finished()

            if self.game.is_player_dead():
                self.game.respawn_player()
                self.reward_gen.respawn_pos(self.game.get_game_variable(GameVariable.HEALTH), \
                                            self.game.get_game_variable(GameVariable.SELECTED_WEAPON_AMMO), \
                                            self.game.get_game_variable(GameVariable.POSITION_X),\
                                            self.game.get_game_variable(GameVariable.POSITION_Y))
            
            step += 1
                
        print("----------TEST at %d step-------------"%(frames))
        print("FRAG:",self.game.get_game_variable(GameVariable.FRAGCOUNT),"DEATH:",self.game.get_game_variable(GameVariable.DEATHCOUNT)-self.pre_death)
        print("REWARD",self.reward_gen.total_reward)
        print("DETAIL:",self.reward_gen.total_reward_detail)
        self.pre_death = self.game.get_game_variable(GameVariable.DEATHCOUNT)

In [3]:
class NetworkSetting:
    
    def state():
        name = "STATE"
        shape = [None,RESOLUTION[0],RESOLUTION[1],RESOLUTION[2]]
        return tf.placeholder(tf.float32,shape=shape,name=name)
    
    def conv1(pre_layer):
        num_outputs = 8
        kernel_size = [6,6]
        stride = 3
        padding = 'SAME'
        activation = tf.nn.relu
        weights_init = tf.contrib.layers.xavier_initializer_conv2d()
        bias_init = tf.constant_initializer(0.1)
        
        return tf.contrib.layers.conv2d(pre_layer,kernel_size=kernel_size,num_outputs=num_outputs, \
                                            stride=stride,padding=padding,activation_fn=activation, \
                                           weights_initializer=weights_init, \
                                            biases_initializer=bias_init)
    def maxpool1(pre_layer):
        return tf.nn.max_pool(pre_layer,[1,3,3,1],[1,2,2,1],'SAME')
    
    def conv2(pre_layer):
        num_outputs = 16
        kernel_size = [3,3]
        stride = 2
        padding = 'SAME'
        activation = tf.nn.relu
        weights_init = tf.contrib.layers.xavier_initializer_conv2d()
        bias_init = tf.constant_initializer(0.1)
        return tf.contrib.layers.conv2d(pre_layer,kernel_size=kernel_size,num_outputs=num_outputs, \
                                            stride=stride,padding=padding,activation_fn=activation, \
                                           weights_initializer=weights_init,biases_initializer=bias_init)
    
    def maxpool2(pre_layer):
        return tf.nn.max_pool(pre_layer,[1,3,3,1],[1,2,2,1],'SAME')
        
    def reshape(pre_layer):
        return tf.contrib.layers.flatten(pre_layer)
        
    def fc1(pre_layer):
        num_outputs = 512
        activation_fn = tf.nn.relu
        weights_init = tf.contrib.layers.xavier_initializer()
        bias_init = tf.constant_initializer(0.1)
        return tf.contrib.layers.fully_connected(pre_layer,num_outputs=num_outputs,activation_fn=activation_fn,\
                                                    weights_initializer=weights_init, biases_initializer=bias_init)
    
    def policy(pre_layer):
        num_outputs=6
        activation_fn = tf.nn.softmax
        weights_init = tf.contrib.layers.xavier_initializer()
        bias_init = tf.constant_initializer(0.1)
        return tf.contrib.layers.fully_connected(pre_layer,num_outputs=num_outputs,activation_fn=activation_fn,\
                                                    weights_initializer=weights_init, biases_initializer=bias_init)
    def value(pre_layer):
        num_outputs = 1
        activation_fn = None
        weights_init = tf.contrib.layers.xavier_initializer()
        bias_init = tf.constant_initializer(0.1)
        
        return tf.contrib.layers.fully_connected(pre_layer,num_outputs=num_outputs,activation_fn=activation_fn,\
                                                weights_initializer=weights_init, biases_initializer=bias_init)

In [4]:
# --グローバルなTensorFlowのDeep Neural Networkのクラスです　-------
class ParameterServer:
    def __init__(self):
        with tf.variable_scope("parameter_server"):      # スレッド名で重み変数に名前を与え、識別します（Name Space）
            self._build_model()            # ニューラルネットワークの形を決定
            
        with tf.variable_scope("summary"):
            self._build_summary()

        self.weights_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="parameter_server")
        self.optimizer = tf.train.RMSPropOptimizer(LEARNING_RATE, RMSProbDecaly)    # loss関数を最小化していくoptimizerの定義です
        
        self.saver = tf.train.Saver()
        
#         print("-------GLOBAL-------")
#         for w in self.weights_params:
#             print(w)

    def _build_model(self):
            self.state = NetworkSetting.state()
            self.conv1 = NetworkSetting.conv1(self.state)
            self.maxpool1 = NetworkSetting.maxpool1(self.conv1)
            self.conv2 = NetworkSetting.conv2(self.maxpool1)
            self.maxpool2 = NetworkSetting.maxpool2(self.conv2)
            reshape = NetworkSetting.reshape(self.maxpool2)
            fc1 = NetworkSetting.fc1(reshape)

            with tf.variable_scope("policy"):
                self.policy = NetworkSetting.policy(fc1)
            
            with tf.variable_scope("value"):
                self.value = NetworkSetting.value(fc1)
                
            print("---------MODEL SHAPE-------------")
            print(self.state.get_shape())
            print(self.conv1.get_shape())
            print(self.conv2.get_shape())
            print(reshape.get_shape())
            print(fc1.get_shape())
            print(self.policy.get_shape())
            print(self.value.get_shape())
                
    def _build_summary(self):
        
        self.a_t = tf.placeholder(tf.float32, shape=(None, N_ACTION))
        self.r_t = tf.placeholder(tf.float32, shape=(None,1))
        
        log_prob = tf.log(tf.reduce_sum(self.policy * self.a_t, axis=1, keep_dims=True)+1e-10)
        advantage = self.r_t - self.value
        
        loss_policy = log_prob * tf.stop_gradient(advantage)
        loss_value = 0.5 * tf.square(advantage)
        entropy = 0.05 * tf.reduce_sum(self.policy * tf.log(self.policy + 1e-10), axis=1, keep_dims=True)
        
        print(loss_policy.get_shape())
        print(loss_value.get_shape())
        
        tf.summary.scalar('loss_policy',loss_policy[0][0])
        tf.summary.scalar('loss_value', loss_value[0][0])
        tf.summary.scalar('entropy', entropy[0][0])
        
        state_shape = self.state.get_shape()
        conv1_shape = self.conv1.get_shape()
        conv2_shape = self.conv2.get_shape()
        tf.summary.image('state',tf.reshape(self.state,[-1, state_shape[1], state_shape[2], state_shape[3]]),1)
        tf.summary.image('conv1',tf.reshape(self.conv1,[-1, conv1_shape[1], conv1_shape[2], 1]),1)
        tf.summary.image('conv2',tf.reshape(self.conv2,[-1, conv2_shape[1], conv2_shape[2], 1]),1)
        
        self.merged = tf.summary.merge_all()
        self.writer = tf.summary.FileWriter("./logs",SESS.graph)
        
    def write_summary(self,step,s1,a,r):
        m = SESS.run(self.merged,feed_dict={self.state:s1,self.a_t:a,self.r_t:r})
        self.writer.add_summary(m,step)
        
    def save_model(self):
        self.saver.save(SESS, MODEL_PATH)
        
    def load_model(self):
        self.saver.restore(SESS,MODEL_PATH)


In [5]:
class RewardGenerater(object):
    def __init__(self,health,ammo,frag,pos_x,pos_y):

        # Reward
        self.rewards = REWARDS
        self.dist_unit = 6.0
        
        self.origin_x = pos_x
        self.origin_y = pos_y
        
        self.pre_health = health
        self.pre_ammo = ammo
        self.pre_frag = frag

        self.total_reward = 0.0
        self.total_reward_detail = {'living':0.0, 'health_loss':0.0, 'medkit':0.0, 'ammo':0.0, 'frag':0.0, 'dist':0.0, 'suicide': 0.0}

    
    def get_reward(self,health,ammo,frag,pos_x,pos_y):
        
        if abs(health) > 10000:
            health = 100.0

        if self.origin_x == 0 and self.origin_y == 0:
            self.origin_x = pos_x
            self.origin_y = pos_y
        
        self.reward_detail = self.calc_reward(frag-self.pre_frag,0.0, \
                                              health-self.pre_health,\
                                              ammo-self.pre_ammo, \
                                              pos_x-self.origin_x, \
                                              pos_y-self.origin_y)
        self.reward = sum(self.reward_detail.values())

        for k,v in self.reward_detail.items():
            self.total_reward_detail[k] += v
        self.total_reward = sum(self.total_reward_detail.values())

        self.pre_frag = frag
        self.pre_health = health
        self.pre_ammo = ammo
                    
        return (self.reward, self.reward_detail)
    
    def calc_reward(self,m_frag,m_death,m_health,m_ammo,m_posx,m_posy):

        ret_detail = {}

        ret_detail['living'] = self.rewards['living']

        if m_frag >= 0:
            ret_detail['frag'] = (m_frag)*self.rewards['frag']
            ret_detail['suicide'] = 0.0
        else:
            ret_detail['suicide'] = (m_frag*-1)*(self.rewards['suicide'])
            ret_detail['frag'] = 0.0
        
        ret_detail['dist'] = int((math.sqrt((m_posx)**2 + (m_posy)**2))/self.dist_unit) * (self.rewards['dist'] * self.dist_unit)
        
        if m_health > 0:
            ret_detail['medkit'] = self.rewards['medkit']
            ret_detail['health_loss'] = 0.0
        else:
            ret_detail['medkit'] = 0.0
            ret_detail['health_loss'] = (m_health)*self.rewards['health_loss'] * (-1)

        ret_detail['ammo'] = (m_ammo)*self.rewards['ammo'] if m_ammo>0 else 0.0
        
        return ret_detail 
    
    def respawn_pos(self,health,ammo,posx, posy):
        self.origin_x = posx
        self.origin_y = posy
        self.pre_health = health
        self.pre_ammo = ammo

    def new_episode(self,health,ammo,posx,posy):
        self.respawn_pos(health,ammo,posx,posy)
        self.pre_frag = 0

        self.total_reward = 0
        self.total_reward_detail={'living':0.0, 'health_loss':0.0, 'medkit':0.0, 'ammo':0.0, 'frag':0.0, 'dist':0.0, 'suicide': 0.0}
    
    def update_origin(self,pos_x, pos_y):
        self.origin_x = pos_x
        self.origin_y = pos_y


In [6]:
class Network_local(object):
    def __init__(self,name,parameter_server):
        self.name = name
        with tf.variable_scope(self.name):
            self._model()
            self._build_graph(parameter_server)
            
        self.s1 = np.empty(shape=(100,RESOLUTION[0],RESOLUTION[1],RESOLUTION[2]),dtype=np.float32)
        self.s2 = np.empty(shape=(100,RESOLUTION[0],RESOLUTION[1],RESOLUTION[2]),dtype=np.float32)
        self.reward = np.empty(shape=(100,1),dtype=np.float32)
        self.action = np.empty(shape=(100,N_ACTION),dtype=np.float32)
        self.isterminal = np.empty(shape=(100,1),dtype=np.int8)
        self.queue_pointer = 0
        
#         print("-----LOCAL weights---")
#         for w in self.weights_params:
#             print(w)
            
#         print("-----LOCAL grads---")
#         for w in self.grads:
#             print(w)
    
    def _model(self):     # Kerasでネットワークの形を定義します
        
        self.state = NetworkSetting.state()
        self.conv1 = NetworkSetting.conv1(self.state)
        self.maxpool1 = NetworkSetting.maxpool1(self.conv1)
        self.conv2 = NetworkSetting.conv2(self.maxpool1)
        self.maxpool2 = NetworkSetting.maxpool2(self.conv2)
        reshape = NetworkSetting.reshape(self.maxpool2)
        fc1 = NetworkSetting.fc1(reshape)

        with tf.variable_scope("policy"):
            self.policy = NetworkSetting.policy(fc1)

        with tf.variable_scope("value"):
            self.value = NetworkSetting.value(fc1)
            
    def _build_graph(self,parameter_server):
        
        self.a_t = tf.placeholder(tf.float32, shape=(None, N_ACTION))
        self.r_t = tf.placeholder(tf.float32, shape=(None,1))
        
        log_prob = tf.log(tf.reduce_sum(self.policy * self.a_t, axis=1, keep_dims=True)+1e-10)
        advantage = self.r_t - self.value
        
        loss_policy = -log_prob * tf.stop_gradient(advantage)
        loss_value = 0.5 * tf.square(advantage)
        entropy = 0.05 * tf.reduce_sum(self.policy * tf.log(self.policy + 1e-10), axis=1, keep_dims=True)
        self.loss_total = tf.reduce_mean(loss_policy + loss_value + entropy)
        
        self.weights_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.name)
        self.grads = tf.gradients(self.loss_total, self.weights_params)

        self.update_global_weight_params = \
            parameter_server.optimizer.apply_gradients(zip(self.grads, parameter_server.weights_params))

        self.pull_global_weight_params = [l_p.assign(g_p) for l_p,g_p in zip(self.weights_params,parameter_server.weights_params)]

        self.push_local_weight_params = [g_p.assign(l_p) for g_p,l_p in zip(parameter_server.weights_params,self.weights_params)]
        
    def pull_parameter_server(self):
        SESS.run(self.pull_global_weight_params)
    
    def push_parameter_server(self):
        SESS.run(self.push_local_weight_params)
        
    def show_weights(self):
        hoge = SESS.run(self.weights_params)
        for i in range(len(hoge)):
            print(hoge[i])
            
    def update_parameter_server(self):
        if self.queue_pointer > 0:
            s1 = self.s1[0:self.queue_pointer]
            s2 = self.s2[0:self.queue_pointer]
            r = self.reward[0:self.queue_pointer]
            a = self.action[0:self.queue_pointer]
            feed_dict = {self.state: s1,self.a_t:a, self.r_t:r}
            SESS.run(self.update_global_weight_params,feed_dict)
            self.queue_pointer = 0
    
    def predict_value(self,s):
        v = SESS.run(self.value,feed_dict={self.state:s})
        return v        
    
    def predict_policy(self,s):
        feed_dict = {self.state:s}
        prob = SESS.run(self.policy, feed_dict)
        return prob
    
    def train_push(self,s,a,r,s_,isterminal):
        self.s1[self.queue_pointer] = s
        self.s2[self.queue_pointer] = s_
        self.action[self.queue_pointer] = a
        self.reward[self.queue_pointer] = r
        self.isterminal[self.queue_pointer] = isterminal
        self.queue_pointer += 1

In [7]:
class Agent(object):
    def __init__(self,name,network):
        self.name = name
        self.network = network
    
    def test_act(self,s1):
        s1 = np.array([s1])
        action_prob = self.network.predict_policy(s1)[0]

        action = np.zeros((N_ACTION,))
#         action[np.argmax(action_prob)] = 1
        print(action_prob)
        action[np.random.choice(6,p=action_prob)] = 1
        return action.tolist()    

In [8]:
# -- main ここからメイン関数です------------------------------
# M0.global変数の定義と、セッションの開始です
frames = 0              # 全スレッドで共有して使用する総ステップ数
isLearned = False       # 学習が終了したことを示すフラグ
SESS = tf.Session()     # TensorFlowのセッション開始
# SESS = tf_debug.LocalCLIDebugWrapperSession(SESS)
TRAINING = "training"
TESTING = "testing"

# M1.スレッドを作成します
with tf.device("/cpu:0"):
    parameter_server = ParameterServer()    # 全スレッドで共有するパラメータを持つエンティティです
    parameter_server.load_model()

test_env = TestEnvironment("test_env",parameter_server)

# M2.TensorFlowでマルチスレッドを実行します
COORD = tf.train.Coordinator()                  # TensorFlowでマルチスレッドにするための準備です
SESS.run(tf.global_variables_initializer())     # TensorFlowを使う場合、最初に変数初期化をして、実行します

test_env.run()
test_env.game.close()

---------MODEL SHAPE-------------
(?, 120, 180, 3)
(?, 40, 60, 8)
(?, 10, 15, 16)
(?, 640)
(?, 512)
(?, 6)
(?, 1)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
(?, 1)
(?, 1)
INFO:tensorflow:Restoring parameters from ./model_v00/model_v00.ckpt
[0.10511322 0.16300257 0.10228134 0.10451867 0.42326882 0.10181539]
[0.10786572 0.16444926 0.10316297 0.10355605 0.41837704 0.1025889 ]
[0.10530205 0.16251993 0.10120516 0.1054652  0.42311808 0.1023896 ]
[0.10779312 0.1633082  0.10189164 0.10432728 0.41935307 0.10332666]
[0.10779312 0.1633082  0.10189164 0.10432728 0.41935307 0.10332666]
[0.10779312 0.1633082  0.10189164 0.10432728 0.41935307 0.10332666]
[0.11009452 0.15864742 0.10273504 0.10530093 0.41857862 0.10464347]
[0.11009452 0.15864742 0.10273504 0.10530093 0.41857862 0.10464347]
[0.11018267 0.15864106 0.10253786 0.10539284 0.41853958 0.10470598]
[0.107171   0.15877292 0.09931066 0.10651069 0.4262461  0.10198861]
[0.10629213 0.1620997  0.09891194 0.10325118 0.417

[0.14795995 0.13879722 0.12370946 0.13135982 0.32121238 0.13696109]
[0.15055336 0.1463641  0.12420234 0.12621763 0.32387576 0.12878689]
[0.15076381 0.14535876 0.12870355 0.12134066 0.32472938 0.12910384]
[0.15008837 0.14615543 0.12897517 0.11919841 0.31415138 0.14143126]
[0.1461344  0.14778113 0.13451694 0.12185321 0.3134689  0.13624534]
[0.14979233 0.146553   0.12633152 0.12386045 0.30861977 0.144843  ]
[0.15581253 0.15367632 0.12486687 0.12458188 0.29797322 0.14308919]
[0.15644541 0.15184285 0.12124655 0.12613338 0.3014595  0.1428724 ]
[0.1568013  0.15170912 0.11977322 0.12537438 0.3037105  0.14263147]
[0.15353088 0.14970234 0.12486441 0.12197043 0.30976462 0.14016727]
[0.15434562 0.15157056 0.12459498 0.12351724 0.3035637  0.14240786]
[0.1546197  0.14839749 0.12470478 0.12231511 0.30400157 0.14596146]
[0.15195565 0.14907141 0.12531443 0.12470201 0.3024834  0.14647304]
[0.1509434  0.15222311 0.12707402 0.12212661 0.30002204 0.14761081]
[0.15192339 0.1475281  0.12769519 0.12222605 0.3

[0.14527161 0.14224416 0.13055015 0.12498693 0.30975804 0.1471891 ]
[0.14708558 0.14123626 0.1310914  0.12513478 0.31089312 0.14455892]
[0.1474321  0.13759781 0.13255894 0.12383774 0.30971816 0.14885521]
[0.14515172 0.14100543 0.13456456 0.121205   0.30858108 0.14949216]
[0.14336637 0.14202042 0.13138415 0.12133238 0.31219697 0.14969963]
[0.14519577 0.14563651 0.12756214 0.12391363 0.31086147 0.1468305 ]
[0.14627703 0.14245795 0.12893042 0.12260336 0.31429124 0.14543998]
[0.1451378  0.14180025 0.1335051  0.11929945 0.31499043 0.145267  ]
[0.14838302 0.14402652 0.12950782 0.12116302 0.31099337 0.14592624]
[0.1461699  0.14953615 0.1287601  0.1209082  0.3080983  0.1465274 ]
[0.14398122 0.1478308  0.12708917 0.12292701 0.31166807 0.14650372]
[0.14453632 0.15128495 0.13128646 0.12360239 0.29902875 0.15026106]
[0.14182946 0.15347183 0.12841219 0.12093046 0.30689684 0.14845924]
[0.14109343 0.15202202 0.129975   0.12342549 0.30230018 0.15118389]
[0.14099132 0.14685954 0.13313508 0.12212907 0.3

[0.14158563 0.13445514 0.13126123 0.12479971 0.31508172 0.15281653]
[0.14521722 0.13152128 0.12787564 0.1233457  0.32026026 0.1517798 ]
[0.14277796 0.13408919 0.13002338 0.1253071  0.31799814 0.14980419]
[0.14385778 0.13337822 0.12812434 0.12680459 0.315096   0.15273912]
[0.14190644 0.13593361 0.1289409  0.12456438 0.31522927 0.15342544]
[0.14030957 0.13522741 0.12961243 0.12710021 0.31535918 0.15239117]
[0.14093629 0.13509373 0.13022909 0.12527116 0.31916252 0.14930724]
[0.14422455 0.13283214 0.13188381 0.13146989 0.30889383 0.15069583]
[0.14153187 0.12994422 0.13175203 0.13067219 0.31241307 0.15368669]
[0.14363556 0.13441671 0.13216773 0.12713039 0.31237495 0.15027466]
[0.1424983  0.1379446  0.13215394 0.12531091 0.31358436 0.14850782]
[0.1436184  0.13154982 0.135501   0.1267951  0.3134888  0.1490469 ]
[0.14364316 0.13505733 0.1314696  0.12268076 0.31755045 0.14959876]
[0.14175689 0.13586308 0.13133839 0.12438671 0.3167702  0.14988472]
[0.14538598 0.13511774 0.13292378 0.12189525 0.3

[0.15029092 0.15015738 0.1351852  0.12771837 0.2902166  0.14643155]
[0.14777116 0.15049903 0.13495757 0.12485885 0.29610625 0.14580718]
[0.14849444 0.15236646 0.1300719  0.12358492 0.29879618 0.14668614]
[0.14857863 0.1487707  0.13171577 0.12523778 0.30424333 0.14145386]
[0.1512652  0.1528468  0.13053635 0.12603444 0.29702204 0.14229514]
[0.14824796 0.14810488 0.13156612 0.12604354 0.3048295  0.14120805]
[0.15198487 0.1508388  0.12978277 0.12412746 0.30257607 0.14069001]
[0.14622097 0.14278238 0.13197677 0.12974654 0.30675393 0.14251944]
[0.13991593 0.14981389 0.1353481  0.1245895  0.3047673  0.14556536]
[0.14487357 0.13849549 0.1360095  0.12206342 0.31862155 0.13993645]
[0.14467336 0.14355236 0.13639314 0.12351079 0.3105735  0.14129682]
[0.14527555 0.14600006 0.13600452 0.12341176 0.30852452 0.14078356]
[0.1383207  0.13504884 0.13589558 0.12282547 0.328202   0.13970757]
[0.13306259 0.1424177  0.14276598 0.1136891  0.32919773 0.1388669 ]
[0.13569869 0.14896727 0.13931896 0.1182887  0.3

[0.15122978 0.13823068 0.13177806 0.12499551 0.31086323 0.14290279]
[0.14862092 0.13342072 0.13175187 0.1287563  0.31070492 0.14674525]
[0.14450732 0.13685241 0.13569853 0.12682244 0.30899173 0.14712754]
[0.14445493 0.13919635 0.1328242  0.12386435 0.31281012 0.14685003]
[0.14464451 0.14150709 0.12899482 0.12222629 0.3139279  0.14869937]
[0.14488195 0.14125896 0.13039166 0.12416515 0.3116439  0.14765836]
[0.14319374 0.14347945 0.13269396 0.1229097  0.3097678  0.14795531]
[0.14906853 0.1384358  0.12732399 0.12257707 0.31722882 0.14536583]
[0.14604546 0.13683279 0.12761782 0.12267234 0.32151282 0.14531887]
[0.15012616 0.13797921 0.12714966 0.11866246 0.32316118 0.14292137]
[0.15132207 0.13595618 0.12836988 0.12191893 0.3202248  0.14220814]
[0.1517958  0.13377526 0.12583362 0.12383664 0.3174663  0.14729242]
[0.14855677 0.13883783 0.12598507 0.11986301 0.3205858  0.14617157]
[0.14911386 0.14057235 0.12834522 0.12157075 0.31668618 0.14371169]
[0.14505957 0.13922074 0.12618956 0.12122574 0.3